# Upload Specification file in each subject folder:
### From: Frink /HCP_Data/HCP_BedpostData/
### To: Candygram 
http://candygram.neurology.emory.edu:8080/#collection/5c48db6de62914004de6350f/folder/5c743889e62914004d6f974c

In [ ]:
# Ensure girder_client installed:
! pip install girder_client

In [1]:
import girder_client
import sys,os,glob

In [2]:
# Subject Data location:
subjRootDir = "/data/HCP_BedpostData/"

# Source subject nodiff:
diffData = "/T1w/Diffusion/nodif*"

# Source ROIs
dtiSpaceMasks = "addlInfo/%s/DTI_ROIs/*"

In [15]:
# Admin login:
apiUrl="http://candygram.neurology.emory.edu:8080/api/v1"
cbGirder = girder_client.GirderClient(apiUrl=apiUrl)
cbGirder.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [4]:
# Destination of Subject data in DTI Space :
subjDTIPath = "/collection/FG/HCP_TractographyData/SubjectRegistrationResults/DTI_Space"
subjDtiTarget = cbGirder.get("resource/lookup?path=" + subjDTIPath)

In [5]:
# List of Subjects:
subjList = [x for x in os.listdir(subjRootDir) if 'addlInfo' not in x]
print(len(subjList),"Subjects have Bedpost Data")

1009 Subjects have Bedpost Data


# Upload Specification file as metadata for each subject folder:
### Spec defines data visualized parameters and load order

In [18]:
# DTI base data: first: nodif, second: nodif_brain, third: nodif_brain_mask
dtiFilesForPapaya = ['nodif.nii.gz',
                     'nodif_brain.nii.gz',
                     'nodif_brain_mask.nii.gz']

imageList = []

In [7]:
s = '100206'
s

'100206'

In [20]:
# Iterate through subjects:
for s in ['100206']: #subjList[0:1]:
    print('S: %s' % s)
    papayaSpecDataDict = {}
    papayaSpec={}
    
    # Subject folder:
    grdrTgtSubjFolder = list(cbGirder.listFolder(subjDtiTarget['_id'],
                                                 name=s))
    # Images in subject folder:
    itemsInSubjFolder = list(cbGirder.listItem(grdrTgtSubjFolder[0]['_id']))
    
    # Subject subfolders: Ex. 'DTI_ROIs'
    foldersInSubjFolder = list(cbGirder.listFolder(grdrTgtSubjFolder[0]['_id']) )
    
    # Iterate through itemsInSubjFolder:
    for i in itemsInSubjFolder:
        # if image in dtiFilesForPapaya, create Specification entry:
        if i['name'] in dtiFilesForPapaya:
            
            # Image location:                    
            imageURL = apiUrl +'/item/' +i['_id'] + '/download?contentDisposition=attachment'
            print(imageURL)
            imageList.append( imageURL )
            
            # Specification Entry:
            papayaSpecDataDict[i['name']] = {'displayName': i['name'].replace('nii.gz',''), 
                                             'grdrFileId': i['_id'],
                                             "alpha": 1, 
                                             "visible": 1}
            
    # Order Specification files:
    for d in dtiFilesForPapaya:
        if d in papayaSpecDataDict:
            papayaSpec.append( papayaSpecDataDict[d]  )
    # Upload metadata to Subject folder:        
    # cbGirder.addMetadataToFolder(grdrTgtSubjFolder[0]['_id'], {"papayaSpec":papayaSpec})
    

S: 100206
http://candygram.neurology.emory.edu:8080/api/v1/item/5c86d6f0e62914004d776c75/download?contentDisposition=attachment
http://candygram.neurology.emory.edu:8080/api/v1/item/5c86d6f0e62914004d776c78/download?contentDisposition=attachment
http://candygram.neurology.emory.edu:8080/api/v1/item/5c86d6f0e62914004d776c72/download?contentDisposition=attachment


AttributeError: 'dict' object has no attribute 'append'

# Define list of images in subject folder metadata: